In [1]:
import pandas as pd

df = pd.read_csv('kkbox_data/kkbox_data.csv')
df.head()

,rank,artist,artist_id,song_id,album_id,date,category
0,1,五月天 (Mayday),9XN-7yg5vg3gYnCdsM,5_DAFKxzMf04hZqXk4,0le7fKde-BNMwoAst3,2021-01-01,mandarin
1,2,盧廣仲 (Crowd Lu),8qCONzjl89Yak9KFxZ,L_Aag3up_NyBpahTJG,4rzv06iAFoOyareMC7,2021-01-01,mandarin
2,3,任然,#,#,#,2021-01-01,mandarin
3,4,艾薇,SkYfuVHgT10-nfv3Tz,XXONe3fIN4kbrmzJqh,XXDPJ346EjhjDZXb-W,2021-01-01,mandarin
4,5,aMEI (張惠妹),Ok6kf9NM0d0grGYNbl,P_c_y1B6adNriKJkm6,Okl8SAlaH9M2s97GAv,2021-01-01,mandarin


In [2]:
album_id = df[df['album_id'] != '#']['album_id'].drop_duplicates()
album_id

0       0le7fKde-BNMwoAst3
1       4rzv06iAFoOyareMC7
3       XXDPJ346EjhjDZXb-W
4       Okl8SAlaH9M2s97GAv
5       D_QwkPGasaFcNVcZCz
               ...        
8788    HYj_0bnwU1EtWSkqVG
8798    9a2zra3DUhQp-Ssz_o
9075    Creg7E-5yWtogi7IJB
9127    GlrhpaVAs-18Awx0VL
9136    -rIWDmb51XBjJ7CTLp
Name: album_id, Length: 368, dtype: object

In [6]:
# get token
from kkbox_developer_sdk.auth_flow import KKBOXOAuth


# Replace CLIENT_ID and CLIENT_SECRET with your ID and Secret
auth = KKBOXOAuth('5e5e2e22f6e9940a106b3befde703767', 'd3aa398ae0aa2018b5205a6a98692a4d')
token = auth.fetch_access_token_by_client_credentials()
access_token = token.access_token

In [7]:
import http.client
import json

conn = http.client.HTTPSConnection("api.kkbox.com")
headers = {
    'accept': "application/json",
    'authorization': f"Bearer {access_token}"
}

In [ ]:
conn.request("GET", f"/v1.1/albums/{id}?territory=TW", headers=headers)

res = conn.getresponse()
data = res.read()

with open('albums_id_data.json', 'w', encoding='utf-8') as f:
    json.dump(json.loads(data), f, ensure_ascii=False, indent=4)
    

In [8]:
from tqdm import tqdm
import time

# List to store the results
results = []

# Iterate through all album IDs
for id in tqdm(album_id):
    try:
        conn.request("GET", f"/v1.1/albums/{id}?territory=TW", headers=headers)
        res = conn.getresponse()
        data = res.read()
        
        # Parse the JSON response
        album_data = json.loads(data.decode("utf-8"))
        
        # Extract the required information
        album_name = album_data.get("name")
        album_id = album_data.get("id")
        release_date =album_data.get("release_date")
        
        # Append to results
        results.append({"name": album_name, "album_id": album_id, "release_date": release_date})
        
        # Add a small delay to avoid overwhelming the API
        time.sleep(0.1)
    
    except Exception as e:
        print(f"Error processing album ID {id}: {str(e)}")

# Create a DataFrame from the results
df_albums = pd.DataFrame(results)

100%|██████████| 368/368 [01:55<00:00,  3.19it/s]

                             name                  id
0                         因為你 所以我  0le7fKde-BNMwoAst3
1                        刻在我心底的名字  4rzv06iAFoOyareMC7
2                           失重前幸福  XXDPJ346EjhjDZXb-W
3                              緩緩  Okl8SAlaH9M2s97GAv
4  如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲  D_QwkPGasaFcNVcZCz


In [9]:
df_albums

,name,id
0,因為你 所以我,0le7fKde-BNMwoAst3
1,刻在我心底的名字,4rzv06iAFoOyareMC7
2,失重前幸福,XXDPJ346EjhjDZXb-W
3,緩緩,Okl8SAlaH9M2s97GAv
4,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,D_QwkPGasaFcNVcZCz
...,...,...
363,Easy,HYj_0bnwU1EtWSkqVG
364,Say So,9a2zra3DUhQp-Ssz_o
365,Sorry,Creg7E-5yWtogi7IJB
366,我唱不了嘻哈,GlrhpaVAs-18Awx0VL
